In [1]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

input_texts = []
labels = []

for label,f in enumerate(input_files):
    print(f"{f} corresponds to label {label}")
    
    for line in open(f):
        line = line.rstrip().lower()
        if line:
            line.translate(str.maketrans('','',string.punctuation))
            
            input_texts.append(line)
            labels.append(label)
            
X_train, X_test, Y_train, Y_test = train_test_split(labels)

2024-01-13 14:39:39,586 [3113] WARNING  py.warnings: /Users/teaching/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


ValueError: not enough values to unpack (expected 4, got 2)

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

class markov_models:
    def __init__(self,input_files):
        self.input_files = input_files
        self.idx = 0
        self.word2idx = {'<unk>':0}
        
    def extract(self):
        input_texts = []
        labels = []
        for label,f in enumerate(self.input_files):
            print(f)
            for line in open(f):
                line = line.rstrip().lower()
                if line:
                    line.translate(str.maketrans('','',string.punctuation))
                    input_texts.append(line)
                    labels.append(label)
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_texts,labels)
        return Xtrain,Xtest,Ytrain,Ytest
    
    def makeWord2idx(self,txt):
        for text in txt:
            tokens = text.split()
            for token in tokens:
                if token not in self.word2idx:
                    self.word2idx[token] = self.idx
                    self.idx+=1
                    
    def transform(self,X):
        if len(self.word2idx.keys())==1:
            self.makeWord2idx(X)
            self.V = len(self.word2idx)
            self.A0 = np.ones((self.V,self.V))
            self.pi0 = np.ones(self.V)
        
            self.A1 = np.ones((self.V,self.V))
            self.pi1 = np.ones(self.V)
        
        Xint = []
        for text in X:
            tokens = text.split()
            line_as_int = [self.word2idx[token] for token in tokens]
            Xint.append(line_as_int)
        return Xint
    
    def compute_matrices(self,text_as_int,A,pi):
        for tokens in text_as_int:
            last_idx = None
            for idx in tokens:
                if last_idx is None:
                    pi[idx]+=1
                else:
                    A[last_idx,idx]+=1
                last_idx = idx
            return A,pi
                
    def load(self):
        Xtrain,Xtest,Ytrain,Ytest = self.extract()
        self.train_int = self.transform(Xtrain)
        self.test_int = self.transform(Xtest)
        
        
        self.A0,self.pi0 = self.compute_matrices([t for t,y in zip(self.train_int,Ytrain) if y==0],self.A0,self.pi0)
        self.A1,self.pi1 = self.compute_matrices([t for t,y in zip(self.train_int,Ytrain) if y==1],self.A1,self.pi1)
        
        self.A0/=self.A0.sum(axis=1,keepdims=True)
        logA0 = np.log(A0)
        self.pi0/=self.pi0.sum()
        logpi0 = np.log(self.pi0)
        
        self.A1/=self.A1.sum(axis=1,keepdims=True)
        logA1 = no.log(self.A1)
        self.pi1/=self.pi1.sum()
        logpi1 = np.log(self.pi1)
        
        #compute priors
        count0 = sum(y==0 for y in Ytrain)
        count1 = sum(y==1 for y in Ytrain)
        total = len(Ytrain)
        self.p0 = count0/total
        self.p1 = count1/total
        self.logp0 = np.log(p0)
        self.logp1 = np.log(p1)
        
class Classifier(markov_models):
    def __init__(self,input_files):
        super().__init__(input_files)
        super().load()
        
    def _compute_log_likelihood(self,inpit_,class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]
        
        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                logprob+=logpi[idx]
            else:
                logprob+=logA[last_idx,idx]
            
            last_idx = idx
        return logprob
    
    def predict(self,inputs):
        predictions = np.zeros(len(inputs))
        for i,input_ in enumerate(inputs):
            posteriors = [self._compute_log_likelihood(input_,c)+self.logpriors[c] for c in range(2)]
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions
    
    def run(self):
        train_pred = self.predict(self.train_int)
        test_pred = self.predict(self.test_int)

In [14]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]
c = Classifier(input_files)
c.run()

edgar_allan_poe.txt
robert_frost.txt


KeyError: 'ride.'